In [88]:
#IMPORTS
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#from dataUtils import *
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold, cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

import dataUtils as du
import random

In [89]:
path_base = "../Data/"
a = 100 #acceleration in steps
hz = 30
samples = 3 #steps in gridsearch
stepsize = 20
full_path = ""#path_base + "phantom_data_20step_markerAndS1Correct.csv" #"phantom_data_20step_markersAndS1.csv"
num_meta_rows = 6

#load all data
df = du.load_dataset(a,samples,stepsize,path_base,full_path)
#print(df.head)

# Change rotations to accurate coordinate system and make translations relative to frame 0
df = du.fixCoordinates(df)
#print(df.head)

#get valid frames (frames without wobeling/ocscilation)
start_offset =  65#80 #(frame before action)
wanted_frames = samples ** 6 #amount of frames to extract
frame_offset = 2*hz #time between moves
spacing = 10 #how many previous frames that have to be still
t_eps = 0.5 #translational error
r_eps =0.5 #rotational error

df_valid, valid_idx = du.getValidFrames(df,wanted_frames,frame_offset,start_offset,spacing,t_eps,r_eps)

#comb_list = #np.load("../../Arduino/20shuffled_targets.npy")
#load targets
comb_list = du.getGridsearchCableLengths(stepsize)
comb_list_rev = []#list(map(list,combinations))[::-1]

#for i in comb_list:
#  comb_list_rev.append(i[::-1])

comb_list = du.getValidTargets(comb_list,valid_idx)

seed = 42

TOTAL FRAMES: 729 (729)
INVALID FRAMES: 0, VALID FRAMES: 729


In [90]:
print(valid_idx)
print(comb_list[:5])
print(comb_list_rev[:5])

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 220, 221,

In [91]:
print(df_valid.max())
print(df_valid.min())

Frame      43805.000000
Time        1460.166667
X_rot          7.778208
Y_rot         10.491430
Z_rot          7.510272
X_trans        9.275269
Y_trans        5.542541
Z_trans       14.873856
dtype: float64
Frame      125.000000
Time         4.166667
X_rot       -6.316861
Y_rot      -10.544286
Z_rot       -7.188015
X_trans     -8.202880
Y_trans     -5.530762
Z_trans    -11.188171
dtype: float64


In [92]:
#Regularize the input data
col = ["X_trans", "Y_trans", "Z_trans", "X_rot", "Y_rot", "Z_rot"]

#based
X_norm = df_valid[col].copy()



#for i in range(6):
#  X_norm[col[i]] = 2 * (X_norm[col[i]] - X_norm[col[i]].min()) / (X_norm[col[i]].max() - X_norm[col[i]].min()) - 1

# #print(X_norm)
print(X_norm.max())
print(X_norm.min())
#print(df_valid.min())
#X_norm = (X - min_vals) / (max_vals-min_vals)

Y = np.array(comb_list)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_norm.values, Y, test_size=0.15, random_state=seed)

X_trans     9.275269
Y_trans     5.542541
Z_trans    14.873856
X_rot       7.778208
Y_rot      10.491430
Z_rot       7.510272
dtype: float64
X_trans    -8.202880
Y_trans    -5.530762
Z_trans   -11.188171
X_rot      -6.316861
Y_rot     -10.544286
Z_rot      -7.188015
dtype: float64


In [93]:
from scipy.spatial.transform import Rotation as R

seed = 42
# Assuming df is your pandas DataFrame
# Normalize translations in DF
col = [ "X_trans", "Y_trans", "Z_trans"]
X_norm = df_valid.copy()
for c in col:
  X_norm[c] = 2 * (X_norm[c] - X_norm[c].min()) / (X_norm[c].max() - X_norm[c].min()) - 1
# Selecting features from column index 2 to 7 (exclusive of 8)
X =X_norm.iloc[:, 2:8].values
print(X_norm.shape)

print(X_norm.columns)
rotations = X[:, :3]
translations = X[:, 3:]
print(np.min(translations))
# Define the positions of 4 points spaced around the original pivot point
# These positions are relative to the pivot point before rotation
point_offsets = np.array([[1, 0, 1],  # Point 1
                           [0, 1, -1],  # Point 2
                           [-1, -1, 0], # Point 3
                           [-1, 0, -1]])# Point 4

# Convert Euler angles to rotation matrices
rot_matrices = R.from_euler('xyz', rotations, degrees=True).as_matrix()

# Apply rotation to the point offsets
rotated_points = np.matmul(rot_matrices, point_offsets.transpose())
rotated_points = np.transpose(rotated_points, axes=(0, 2, 1))


for i in range(rotated_points.shape[0]):
  for point in rotated_points[i]:
    point[0] += translations[i,0]
    point[1] += translations[i,1]
    point[2] += translations[i,2]

rotated_points = rotated_points.reshape(-1,12)

X =rotated_points#np.concatenate((rotated_points,translations),axis = 1)#rotated_points# # #np.concatenate((rotated_points, translations), axis=1)
# 12 features, xyz xyz xyz xyz
min_xs, max_xs = min([np.min(X[:,0]),np.min(X[:,3]),np.min(X[:,6]),np.min(X[:,9])]), max([np.max(X[:,0]),np.max(X[:,3]),np.max(X[:,6]),np.max(X[:,9])])
min_ys, max_ys = min([np.min(X[:,1]),np.min(X[:,4]),np.min(X[:,7]),np.min(X[:,10])]), max([np.max(X[:,1]),np.max(X[:,4]),np.max(X[:,7]),np.max(X[:,10])])
min_zs, max_zs = min([np.min(X[:,2]),np.min(X[:,5]),np.min(X[:,8]),np.min(X[:,11])]), max([np.max(X[:,2]),np.max(X[:,5]),np.max(X[:,8]),np.max(X[:,11])])

#print(min_xs,max_xs)
#print(min_ys,max_ys)
#print(min_zs,max_zs)
# print(X.shape)

# for i in range(X.shape[0]):
#   for x in [0,3,6,9]:
#     X[i,x] = X[i,x] / np.sqrt(2)
#   for y in [1,4,7,10]:
#     X[i,y] = X[i,y] / np.sqrt(2)
#   for z in [2,5,8,11]:
#     X[i,z] = X[i,z] / np.sqrt(2)

#normalize all vectors based on their max/min x,y or z value
for i in range(X.shape[0]):
  for x in [0,3,6,9]:
    X[i,x] = 2*((X[i,x]-min_xs)/(max_xs-min_xs)) - 1
  for y in [1,4,7,10]:
    X[i,y] = 2*((X[i,y]-min_ys)/(max_ys-min_ys)) -1
  for z in [2,5,8,11]:
    X[i,z] = 2*((X[i,z]-min_zs)/(max_zs-min_zs)) -1



print(X.shape)
print(np.min(X,axis=(0)),np.max(X,axis=(0)))
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=seed)

(729, 8)
Index(['Frame', 'Time', 'X_rot', 'Y_rot', 'Z_rot', 'X_trans', 'Y_trans',
       'Z_trans'],
      dtype='object')
-1.0
(729, 12)
[ 0.02895873 -0.50022303  0.01300751 -0.52122643  0.00887498 -0.95015703
 -0.97769996 -1.         -0.54041494 -1.         -0.494009   -1.        ] [ 1.          0.50944342  1.          0.56467111  1.         -0.0066333
  0.01849922  0.00539313  0.49088166  0.04820136  0.50044761 -0.0105011 ]


In [94]:
# Assuming df is your pandas DataFrame
# Selecting features from column index 2 to 7 (exclusive of 8)
#X = df_valid.iloc[:, 2:8].values

# Assuming your output targets are in the columns from index 8 onwards
#Y = np.array(comb_list)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.15, random_state=seed)

# Initialize the RandomForestRegressor
random_forest = RandomForestRegressor(n_estimators=100, random_state=seed)

# Specify the number of folds for cross-validation
k_folds = 5

# Initialize the KFold cross-validator
kf = KFold(n_splits=k_folds, shuffle=True, random_state=seed)

# Initialize list to store validation MSE scores
validation_mse_scores = []

# Iterate over each fold
for train_index, val_index in kf.split(X_train):
    # Split the data into training and validation sets for this fold
    X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]
    
    # Fit the model on the training data
    random_forest.fit(X_train_fold, y_train_fold)
    
    # Predict on the validation set
    y_pred = random_forest.predict(X_val_fold)
    
    # Compute MSE for this fold
    mse = mean_squared_error(y_val_fold, y_pred)
    validation_mse_scores.append(mse)

# Print the validation MSE scores
print("Validation MSE scores:", validation_mse_scores)
print("Mean Validation MSE:", np.mean(validation_mse_scores))
print("Std Validation MSE:", np.std(validation_mse_scores))

Validation MSE scores: [86.39096774193547, 94.27123655913978, 94.03112903225808, 90.41688172043011, 94.22596205962058]
Mean Validation MSE: 91.86723542267681
Std Validation MSE: 3.10220186120384


In [95]:
0/0

ZeroDivisionError: division by zero

In [ ]:
class FeedForwardNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(FeedForwardNN, self).__init__()
        #self.bn1 = nn.BatchNorm1d(6)
        self.fc1 = nn.Linear(input_size, hidden_size)
        #self.bn2 = nn.BatchNorm1d(30)
        self.dropout1 = nn.Dropout(p=0.1)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        #self.bn3 = nn.BatchNorm1d(30)
        self.dropout2 = nn.Dropout(p=0.1)
        self.fc3 = nn.Linear(hidden_size, hidden_size)
        #self.bn4 = nn.BatchNorm1d(30)
        self.dropout3 = nn.Dropout(p=0.1)
        self.fc4 = nn.Linear(hidden_size, output_size)

        #self.fc5 = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.fc1(x))
        #x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        #x = self.dropout2(x)
        x = self.relu(self.fc3(x))
        #x = self.dropout3(x)
        x = self.fc4(x)
        return x

# Define input and output sizes
input_size = 6
output_size = 6
hidden_size = 32#64
batch_size = 4
lr = 0.001
num_epochs = 500

# Create the model
model = FeedForwardNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

In [ ]:
#SETUP DATALOADERS
from torch.utils.data import TensorDataset, DataLoader

def create_data_loaders(train_index, val_index):
    X_train_fold,y_train_fold = torch.tensor(X_train[train_index], dtype=torch.float32), torch.tensor(y_train[train_index], dtype=torch.float32)
    X_val_fold,y_val_fold = torch.tensor(X_train[val_index], dtype=torch.float32), torch.tensor(y_train[val_index], dtype=torch.float32)
    train_dataset = TensorDataset(X_train_fold,y_train_fold)
    val_dataset = TensorDataset(X_val_fold,y_val_fold)
    
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    
    return train_loader, val_loader

# train_data_tensor = torch.tensor(X_train, dtype=torch.float32)
# targets_tensor = torch.tensor(y_train, dtype=torch.float32)
# val_data_tensor = torch.tensor(X_test, dtype=torch.float32)
# val_targets_tensor = torch.tensor(y_test, dtype=torch.float32)

# # Combine the input data and target values into a TensorDataset
# train_dataset = TensorDataset(train_data_tensor, targets_tensor)
# test_dataset = TensorDataset(val_data_tensor, val_targets_tensor)

# # Create a DataLoader for batch processing
# batch_size = 4
# train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
# val_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
# #TRAINING
# # Lists to store training and validation losses
# train_losses = []
# val_losses = []
# i = 0
# for train_index, val_index in kf.split(X_train):


#     # Initialize early stopping parameters
#     # best_val_loss = np.Inf
#     # patience = 15
#     # counter = 0
#     # Create the model
#     model = FeedForwardNN(input_size, hidden_size, output_size)

#     # Define loss function and optimizer
#     criterion = nn.MSELoss()
#     optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed
#     train_losses_fold = []
#     val_losses_fold = []
#     train_loader,val_loader = create_data_loaders(train_index, val_index)

#     for epoch in range(num_epochs):
#         model.train()  # Set the model to training mode
#         running_train_loss = 0.0
        
#         # Training loop
#         for inputs, targets in train_loader:
#             optimizer.zero_grad()
#             outputs = model(inputs)
#             loss = criterion(outputs, targets)
#             loss.backward()
#             optimizer.step()
#             running_train_loss += loss.item()
        
#         # Compute average training loss for the epoch
#         average_train_loss = running_train_loss / len(train_loader)
#         train_losses_fold.append(average_train_loss)
        
#         # Validation loop
#         model.eval()  # Set the model to evaluation mode
#         running_val_loss = 0.0
#         with torch.no_grad():
#             for inputs, targets in val_loader:
#                 outputs = model(inputs)
#                 loss = criterion(outputs, targets)
#                 running_val_loss += loss.item()
        
#         # Compute average validation loss for the epoch
#         average_val_loss = running_val_loss / len(val_loader)
#         val_losses_fold.append(average_val_loss)

#          # Check for improvement in validation loss
#         # if running_val_loss < best_val_loss:
#         #     best_val_loss = running_val_loss
#         #     counter = 0
#         #     # Save the best model state if needed
#         #     # torch.save(model.state_dict(), 'best_model.pt')
#         # else:
#         #     counter += 1
#         #     # If no improvement for patience epochs, stop training
#         #     if counter >= patience:
#         #         print(f'Early stopping at epoch {epoch}.')
#         #         break
        
#         # Print training and validation loss for the epoch
#         print(f"Fold: {i}, Epoch [{epoch+1}/{num_epochs}], Train Loss: {average_train_loss:.4f}, Val Loss: {average_val_loss:.4f}", end = "\r")
#     i += 1
#     train_losses.append(train_losses_fold)
#     val_losses.append(val_losses_fold)

# # Plot the training and validation losses
# plt.figure()
# for i in range(k_folds):
#     plt.plot(train_losses[i], label='Train Loss')
#     plt.plot(val_losses[i], label='Val Loss')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss')
# plt.legend()
# plt.show()

In [ ]:
#x[-1] for x in 

In [ ]:
# # (5,100)
# #print(np.array(train_losses).shape)
# for i in range(5):
#   print(val_losses[i][-1])
# # print(train_losses[:][1])
# # print(train_losses[1])
# # print(train_losses)

In [ ]:
# plt.figure()
# for i in range(k_folds):
#     plt.plot(train_losses[i], label=f'Train Loss fold {i}')
# plt.xlabel('Epoch')
# plt.ylabel('Loss')
# plt.title('Training and Validation Loss')
# plt.legend()
# plt.grid()
# plt.show()

# plt.figure()
# for i in range(k_folds):
#     plt.plot(val_losses[i], label=f"Val Loss fold {i}")
# plt.xlabel('Epoch')
# plt.ylabel('MSE')
# plt.title('Training and Validation Loss')
# plt.legend()
# plt.grid()
# plt.show()
# print(f"Mean Training MSE: {np.mean(np.array(train_losses)[:,-1])}, Mean Training STD: {np.std(np.array(train_losses)[:,-1])}")
# print(f"Mean Validation MSE: {np.mean(np.array(val_losses)[:,-1])}, Mean Validation STD: {np.std(np.array(val_losses)[:,-1])}")

In [ ]:
# np.save("data/20_new/3hid_64_neu_500epoch_p001lr_norm_trainloss",np.array(train_losses))
# np.save("data/20_new/3hid_64_neu_500epoch_p001lr_norm_valloss",np.array(val_losses))

In [ ]:
# Lists to store training and validation losses

# Create the model
model = FeedForwardNN(input_size, hidden_size, output_size)



# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Adjust learning rate as needed

train_data_tensor = torch.tensor(X_train, dtype=torch.float32)
targets_tensor = torch.tensor(y_train, dtype=torch.float32)
val_data_tensor = torch.tensor(X_test, dtype=torch.float32)
val_targets_tensor = torch.tensor(y_test, dtype=torch.float32)
# Combine the input data and target values into a TensorDataset
train_dataset = TensorDataset(train_data_tensor, targets_tensor)
test_dataset = TensorDataset(val_data_tensor, val_targets_tensor)

# Create a DataLoader for batch processing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

train_losses = []
val_losses = []

for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    running_train_loss = 0.0
    
    # Training loop
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        running_train_loss += loss.item()
    
    # Compute average training loss for the epoch
    average_train_loss = running_train_loss / len(train_loader)
    train_losses.append(average_train_loss)
    
    # Validation loop
    model.eval()  # Set the model to evaluation mode
    running_val_loss = 0.0
    with torch.no_grad():
        for inputs, targets in val_loader:
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            running_val_loss += loss.item()
    
    # Compute average validation loss for the epoch
    average_val_loss = running_val_loss / len(val_loader)
    val_losses.append(average_val_loss)
    
    # Print training and validation loss for the epoch
    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {average_train_loss:.4f}, Val Loss: {average_val_loss:.4f}", end = "\r")

In [ ]:
# #SAVE MODEL
#model_name = "3hid_32_neu_500epoch_p001lr_norm-11_rev20"
#torch.save(model, "models/"+ model_name + ".pth")